In [15]:
import pandas as pd

df = pd.read_csv("employee_churn_data_clearned.csv")
df

,department_0,department_1,department_2,department_3,department_4,department_5,department_6,department_7,department_8,department_9,promoted,review,projects,salary,tenure,satisfaction,bonus,avg_hrs_month,left
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.0,3,0,5.0,3.0,0,0.0,0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3.0,4,2,8.0,1.0,0,2.0,0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3.0,3,2,5.0,3.0,1,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,3.0,2,1,5.0,3.0,1,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1.0,4,2,5.0,4.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10489,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4.0,3,1,6.0,0.0,0,0.0,1
10490,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3.0,3,1,6.0,0.0,0,1.0,1
10491,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4.0,4,1,8.0,4.0,0,2.0,1
10492,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3.0,3,1,6.0,0.0,0,1.0,1


In [16]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.3, random_state=0)
valid_data, test_data = train_test_split(test_data, test_size=0.5, random_state=0)
train_data.shape[0], valid_data.shape[0], test_data.shape[0]

(7345, 1574, 1575)

In [17]:
X_train = train_data.drop(columns= 'left', axis = 'columns')
y_train = train_data['left']


In [6]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import KBinsDiscretizer, StandardScaler, OrdinalEncoder
from sklearn.feature_extraction.text import TfidfVectorizer as TfidfVec
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

def create_pipeline():
    
    grid = {
        "n_neighbors": [1, 5, 10, 20],
        "algorithm": ['auto'],
        "leaf_size": [10],
        "p": [0.01, 0.1],
    }
    gcv_knb = GridSearchCV(KNeighborsClassifier(), grid, n_jobs=-1)

    grid = {
        "gamma": ["scale", "auto"],
    }
    gcv_svc = GridSearchCV(SVC(), grid, n_jobs=-1)

    # estimators = [('gcv_svc', gcv_svc)]
    estimators = [('gcv_knb', gcv_knb), ('gcv_svc', gcv_svc)]
    return make_pipeline(
        make_column_transformer(
            (HashingVectorizer(n_features= 10, binary= True), ["department"]),
            (StandardScaler(), ['avg_hrs_month']),
            (KBinsDiscretizer(n_bins= 5), ["review", "satisfaction", "avg_hrs_month"]),
            (OrdinalEncoder(), ['salary'])
        ),
        StackingClassifier(estimators=estimators, stack_method='predict')
    )

pipeline = create_pipeline()
pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('hashingvectorizer',
                                                  HashingVectorizer(binary=True,
                                                                    n_features=10),
                                                  ['department']),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  ['avg_hrs_month']),
                                                 ('kbinsdiscretizer',
                                                  KBinsDiscretizer(),
                                                  ['review', 'satisfaction',
                                                   'avg_hrs_month']),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['salary'])])),
                ('stackingclassifier',
                 StackingClassifier(estimators=[('gcv_knb',
                                                 GridSearchCV(estimator=KNeighborsClassifier(),
                                                              n_jobs=-1,
                                                              param_grid={'algorithm': ['auto'],
                                                                          'leaf_size': [10],
                                                                          'n_neighbors': [1,
                                                                                          5,
                                                                                          10,
                                                                                          20],
                                                                          'p': [0.01,
                                                                                0.1]})),
                                                ('gcv_svc',
                                                 GridSearchCV(estimator=SVC(),
                                                              n_jobs=-1,
                                                              param_grid={'gamma': ['scale',
                                                                                    'auto']}))],
                                    stack_method='predict'))])

In [19]:
pipe = pipeline.fit_transform(X_train,y_train)


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("employee_churn_data.csv")
df

,department,promoted,review,projects,salary,tenure,satisfaction,bonus,avg_hrs_month,left
0,operations,0,0.577569,3,low,5.0,0.626759,0,180.866070,no
1,operations,0,0.751900,3,medium,6.0,0.443679,0,182.708149,no
2,support,0,0.722548,3,medium,6.0,0.446823,0,184.416084,no
3,logistics,0,0.675158,4,high,8.0,0.440139,0,188.707545,no
4,sales,0,0.676203,3,high,5.0,0.577607,1,179.821083,no
...,...,...,...,...,...,...,...,...,...,...
9535,operations,0,0.610988,4,medium,8.0,0.543641,0,188.155738,yes
9536,logistics,0,0.746887,3,medium,8.0,0.549048,0,188.176164,yes
9537,operations,0,0.557980,3,low,7.0,0.705425,0,186.531008,yes
9538,IT,0,0.584446,4,medium,8.0,0.607287,1,187.641370,yes


In [9]:
salary_lvl = [['low','medium','high']]

OrdinalEncoder(categories=salary_lvl).fit_transform(df['salary'].to_numpy().reshape(-1, 1))[:5]

array([[1.],
       [2.],
       [2.],
       [0.],
       [0.]])